In [1]:
pip install tensorflow keras pandas numpy jupyter notebook tqdm opencv-contrib-python scikit-learn

In [ ]:
pip uninstall keras


In [ ]:
pip install keras


In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [ ]:
pip uninstall keras tensorflow

In [2]:
# from keras.utils import to_categorical
# from keras_preprocessing.image import load_img
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# import os
# import pandas as pd
# import numpy as np



from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os

In [3]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    print("1")
    for label in os.listdir(dir):
        print("2")
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [5]:
import pandas as pd

In [6]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

1
2
angry completed
2
disgust completed
2
fear completed
2
happy completed
2
neutral completed
2
sad completed
2
surprise completed


In [7]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [8]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

1
2
angry completed
2
disgust completed
2
fear completed
2
happy completed
2
neutral completed
2
sad completed
2
surprise completed


In [9]:
print(test)
print(test['image'])

                                    image     label
0       images/validation\angry\10052.jpg     angry
1       images/validation\angry\10065.jpg     angry
2       images/validation\angry\10079.jpg     angry
3       images/validation\angry\10095.jpg     angry
4       images/validation\angry\10121.jpg     angry
...                                   ...       ...
7061  images/validation\surprise\9806.jpg  surprise
7062  images/validation\surprise\9830.jpg  surprise
7063  images/validation\surprise\9853.jpg  surprise
7064  images/validation\surprise\9878.jpg  surprise
7065   images/validation\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/validation\angry\10052.jpg
1         images/validation\angry\10065.jpg
2         images/validation\angry\10079.jpg
3         images/validation\angry\10095.jpg
4         images/validation\angry\10121.jpg
                       ...                 
7061    images/validation\surprise\9806.jpg
7062    images/validation\surprise\9830.jpg

In [10]:
from tqdm.notebook import tqdm

In [11]:
# def extract_features(images):
#     features = []
#     for image in tqdm(images):
#         img = load_img(image,grayscale =  True )
#         img = np.array(img)
#         features.append(img)
#     features = np.array(features)
#     features = features.reshape(len(features),48,48,1)
#     return features
    
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [12]:
import numpy as np

train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

In [13]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [20]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [22]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 212s 922ms/step - accuracy: 0.2346 - loss: 1.8370 - val_accuracy: 0.2573 - val_loss: 1.7889
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 221s 977ms/step - accuracy: 0.2585 - loss: 1.7808 - val_accuracy: 0.3438 - val_loss: 1.6518
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 209s 923ms/step - accuracy: 0.3375 - loss: 1.6457 - val_accuracy: 0.4462 - val_loss: 1.4459
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 200s 884ms/step - accuracy: 0.4035 - loss: 1.5114 - val_accuracy: 0.4817 - val_loss: 1.3578
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 200s 885ms/step - accuracy: 0.4515 - loss: 1.4238 - val_accuracy: 0.4960 - val_loss: 1.3187
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 199s 878ms/step - accuracy: 0.4712 - loss: 1.3758 - val_accuracy: 0.4924 - val_loss: 1.3062
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 198s 877ms/step - accuracy: 0.4858 - loss: 1.3351 - val_accuracy: 0.5348 - val_loss: 1.2277
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 198s 877ms/step - accuracy: 

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
# json_file = open("facialemotionmodel.json", "r")
# model_json = json_file.read()
# json_file.close()
# model = model_from_json(model_json)
# model.load_weights("facialemotionmodel.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [ ]:
from PIL import Image

def ef(image):
    img = Image.open(image).convert('L')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature

In [ ]:
# # # image = 'images\validation\sad\312.jpg'
# # # print("original image is of sad")
# # # img = ef(image)
# # # pred = model.predict(img)
# # # pred_label = label[pred.argmax()]
# # # print("model prediction is ",pred_label)

# # image = 'images\validation\sad\312.jpg'
# # print("original image is of sad")
# # img = ef(image)
# # pred = model.predict(img)
# # pred_label = label[pred.argmax()]
# # print("model prediction is ",pred_label)

# import numpy as np
# from PIL import Image

# # Define constants
# IMAGE_PATH = 'images/validation/sad/312.jpg'
# LABELS = [...]  # Define your labels here

# def load_and_preprocess_image(image_path):
#     try:
#         img = Image.open(image_path).convert('L')
#         feature = np.array(img)
#         feature = feature.reshape(1, 48, 48, 1)
#         return feature
#     except Exception as e:
#         print(f"Error loading and preprocessing image: {e}")
#         return None

# def main():
#     image_path = IMAGE_PATH
#     print("Original image is of sad")
#     preprocessed_image = load_and_preprocess_image(image_path)
#     if preprocessed_image is not None:
#         try:
#             pred = model.predict(preprocessed_image)
#             pred_label = LABELS[pred.argmax()]
#             print("Model prediction is ", pred_label)
#         except Exception as e:
#             print(f"Error making prediction: {e}")

# if __name__ == "__main__":
#     main()

import numpy as np
from PIL import Image

# Define constants
IMAGE_PATH = 'images/validation/sad/312.jpg'
LABELS = [...]  # Define your labels here

def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('L')
        feature = np.array(img)
        feature = feature.reshape(1, 48, 48, 1)
        return feature
    except Exception as e:
        print(f"Error loading and preprocessing image: {e}")
        return None

def main():
    image_path = IMAGE_PATH
    print("Original image is of sad")
    preprocessed_image = load_and_preprocess_image(image_path)
    if preprocessed_image is not None:
        try:
            pred = model.predict(preprocessed_image)
            pred_index = pred.argmax()
            print("Model prediction confidence scores:", pred)
            print("Model prediction index:", pred_index)
            if pred_index < len(LABELS):
                pred_label = LABELS[pred_index]
                print("Model prediction is", pred_label)
            else:
                print(f"Error: model prediction index {pred_index} is out of range of labels ({len(LABELS)})")
        except Exception as e:
            print(f"Error making prediction: {e}")

if __name__ == "__main__":
    main()

In [ ]:
pip install matplotlib


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = 'images\validation\sad\35411.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')